In [1]:
# Contraction Hierarchy

from sys import getsizeof
from time import process_time
import osmnx
import networkx as nx
from search_optimization_tools.routing import cost, draw_route
from search_optimization_tools.structures import Node, Solution

osmnx.utils.config(requests_kwargs={"verify": False})
reference = (43.661667, -79.395)
G = osmnx.graph_from_point(reference, dist=300, clean_periphery=True, simplify=True)

origin =55808290
destination = 389677909


In [2]:
# Remove parallel edges, self-loops

clean_G = nx.DiGraph()
clean_G.clear_edges()
orig_G = G.copy()
for u in G.nodes:
    for v in G[u]:
        if u == v: G.remove_edge(u,v); continue
        if len(G[u][v]) > 1:
            keep = min(G[u][v], key=lambda x:G[u][v][x]['length'])
            keep = G[u][v][keep]
            clean_G.add_edge(u,v,**keep)
            continue
        clean_G.add_edge(u,v,**G[u][v][0])

G = clean_G


In [3]:
start_time = process_time()
shortcuts = {}
shortest_paths = dict(nx.all_pairs_dijkstra_path_length(G, weight="length"))
current_G = G.copy()
for node in G.nodes:
    current_G.remove_node(node)
    current_shortest_paths = dict(
        nx.all_pairs_dijkstra_path_length(current_G, weight="length")
    )
    for u in current_shortest_paths:
        if u == node:
            continue
        SP_contracted = current_shortest_paths[u]
        SP_original = shortest_paths[u]
        for v in SP_contracted:
            if u == v or node == v:
                continue
            if (
                SP_contracted[v] != SP_original[v]
                and G.has_edge(node, u)
                and G.has_edge(node, v)
            ):
                G.add_edge(u, v, length=SP_original[v], contracted=True)
                shortcuts[(u,v)] = node
end_time = process_time()
preprocess_sln = Solution(len(shortcuts), end_time-start_time, getsizeof(shortest_paths)+getsizeof(current_shortest_paths)+getsizeof(shortcuts),len(G.nodes))


In [4]:
print(f"Shortcut edges added: {preprocess_sln.result}")
print(f"Preprocessing time: {preprocess_sln.time} s")
print(f"Preprocessing Space: {preprocess_sln.space} bytes")
print(f"Nodes Explored: {preprocess_sln.explored}")

Shortcut edges added: 698
Preprocessing time: 19.359375 s
Preprocessing Space: 55712 bytes
Nodes Explored: 404


In [ ]:
def Bidirectional_Dijkstra(origin, destination, unrelaxed_nodes, expand_kwargs = {}):
    time_start = process_time()  # Time tracking
    frontier = deepcopy(unrelaxed_nodes)

    space_required = getsizeof(frontier)

    explored_f = set()
    explored_b = set()

    shortest_dist_f = {node.get_id(): math.inf for node in frontier}
    shortest_dist_b = {node.get_id(): math.inf for node in frontier}

    shortest_dist_f[origin] = 0
    shortest_dist_b[destination] = 0

    found = False
    route = []

    altr_expand = True  # to alternate between front and back

    while frontier and not found:
        if altr_expand:  # Forward
            node = min(frontier, key=lambda node: shortest_dist_f[node.get_id()])
            # relaxing the node, so this node's value in shortest_dist is the shortest distance between the origin and destination
            frontier.remove(node)
            explored_f.add(node)
            # if the destination node has been relaxed then that is the route we want
            if node == destination:
                route = node.path()
                found = True
                continue

            # otherwise, let's relax edges of its neighbours
            for child in node.expand(**expand_kwargs):
                # skip self-loops
                if child.get_id() in explored_f:
                    continue

                # Check the child is collided
                if child in explored_b:
                    overlapped = next((node for node in explored_b if node == child))
                    # we don't take the overlapped node twice
                    route = child.path()[:-1] + overlapped.path()[::-1]
                    found = True
                    break

                child_obj = next(
                    (node for node in frontier if node.get_id() == child.get_id()), None
                )
                child_obj.set_distance(child.get_distance())
                distance = shortest_dist_f[node.get_id()] + child.get_distance()
                if distance < shortest_dist_f[child_obj.get_id()]:
                    shortest_dist_f[child_obj.get_id()] = distance
                    child_obj.set_parent(node)
            altr_expand = False
        if not altr_expand:  # Backward
            node = min(frontier, key=lambda node: shortest_dist_b[node.get_id()])
            # relaxing the node, so this node's value in shortest_dist is the shortest distance between the origin and destination
            frontier.remove(node)
            explored_b.add(node)
            # if the destination node has been relaxed then that is the route we want
            if node == origin:
                route = node.path()[::-1]
                found = True
                continue

            # otherwise, let's relax edges of its neighbours
            for child in node.expand(backwards=True, **expand_kwargs):
                # skip self-loops
                if child.get_id() in explored_b:
                    continue

                # Check the child is collided
                if child in explored_f:
                    overlapped = next(
                        (node for node in explored_f if node == child), None
                    )
                    route = overlapped.path()[:-1] + child.path()[::-1]
                    found = True
                    break

                child_obj = next(
                    (node for node in frontier if node.get_id() == child.get_id()), None
                )
                child_obj.set_distance(child.get_distance())
                distance = shortest_dist_b[node.get_id()] + child.get_distance()
                if distance < shortest_dist_b[child_obj.get_id()]:
                    shortest_dist_b[child_obj.get_id()] = distance
                    child_obj.set_parent(node)
            altr_expand = True
    time_end = process_time()  # Time tracking
    return Solution(
        route, time_end - time_start, space_required, len(explored_f) + len(explored_b)
    )

In [16]:
#from search_optimization_tools.algorithms.graph_search import Bidirectional_Dijkstra
unrelaxed_nodes = [Node(G,osmid) for osmid in G.nodes()]
result = Bidirectional_Dijkstra(origin, destination, unrelaxed_nodes, expand_kwargs={"multigraph": False})
uncontracted_route = [result.result[0]]
for u,v in zip(result.result[:-1],result.result[1:]):
    if (u,v) in shortcuts or (v,u) in shortcuts:
        print(f"Unpacked {u}-{v} into {u}-{shortcuts[(u,v)]}-{v}")
        uncontracted_route.append(shortcuts[(u,v)])
    uncontracted_route.append(v)

Unpacked 389678139-3707407638 into 389678139-389678138-3707407638
Unpacked 3707407638-389678131 into 3707407638-6028562355-389678131
Unpacked 854322047-389677909 into 854322047-123347984-389677909


In [17]:
print(uncontracted_route)

[55808290, 304891685, 1721866234, 9270977970, 3179025274, 9270977966, 389678267, 24960090, 389678273, 24959523, 50885177, 389677947, 2143489692, 2480712846, 389678140, 389678139, 389678138, 3707407638, 6028562355, 389678131, 6028562356, 854322047, 123347984, 389677909]


In [29]:
draw_route(orig_G, uncontracted_route[19:21])

Map(center=[43.6607508, -79.3948913], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…